In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import platform
path = "c:/Windows/Fonts/malgun.ttf" 
from matplotlib import font_manager, rc 
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') 
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

Unknown system... sorry~~~~


In [2]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import re
import time
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm

In [3]:
tokenized_words = pd.read_csv('./HairShopReviewFeatureNames_2020-05-27.csv', index_col=0, encoding='utf-8')

In [4]:
for a in tokenized_words.pos_names[0:500]:
    print(a, end=',') #형태소 단위로 clean_summary 내 모든 단어가 , 단위로 분절됨.
#해당 내용은 names_list.csv 로 저장.

가가,가감,가감 없이,가게,가게 내부,가게 내부 는,가게 너무,가게 는,가게 도,가게 도 깔끔,가게 되,가게 분위기,가게 분위기 도,가게 안,가게 앞,가게 앞 주차,가게 옆,가게 오픈,가게 이,가게 이름,가게 인테리어,가게 인테리어 도,가게 입구,가게 있,가격,가격 가격,가격 가격표,가격 가능,가격 가성,가격 가성 비,가격 감성,가격 감성 담,가격 같,가격 거품,가격 걱정,가격 검색,가격 검색 해,가격 검증,가격 검증 된,가격 고,가격 고 퀄리티,가격 고객,가격 고객 님,가격 고급,가격 고품격,가격 공개,가격 관리,가격 관리 받,가격 괜찮,가격 구매,가격 궁금,가격 궁금 신,가격 그대로,가격 기분,가격 기분 좋,가격 기장,가격 기장 추가,가격 꼼꼼,가격 남자,가격 내,가격 너무,가격 넘,가격 네이버,가격 네이버 맨,가격 네이버 예약,가격 높,가격 높 은,가격 는,가격 다,가격 다양,가격 다양 한,가격 대,가격 대 는,가격 대로,가격 대비,가격 대비 괜찮,가격 대비 너무,가격 대비 만족,가격 대비 서비스,가격 대비 정말,가격 대비 좋,가격 대한,가격 더,가격 도,가격 도 가격,가격 도 곳,가격 도 괜찮,가격 도 굉장히,가격 도 나쁘,가격 도 나와,가격 도 너무,가격 도 넘,가격 도 다,가격 도 더,가격 도 만,가격 도 만족,가격 도 많이,가격 도 맘,가격 도 매우,가격 도 미리,가격 도 미용실,가격 도 부담,가격 도 비싸,가격 도 비싼,가격 도 생각,가격 도 서비스,가격 도 싸,가격 도 아주,가격 도 엄청,가격 도 완전,가격 도 이,가격 도 이렇게,가격 도 있,가격 도 잘,가격 도 저렴,가격 도 적당,가격 도 적혀,가격 도 정말,가격 도 정찰,가격 도 좋,가격 도 중요,가격 도 진짜,가격 도 착하,가격 도 착한,가격 도 착한데,가격 도 착해서,가격 도 착해요,가격 도 참,가격 도 참고,가격 도 타,가격 도 할인,가격 도 합리,가격 도 확인,가격 도착,가격 도착 고,가격 동일,가격 두,가격 두 저렴,가격 듣,가격 듣 고,가격 디자이너,가격 디자인,가격 디자인 상담

In [5]:
#위 df의 names의 형태소 확인 후 추가하고자 하는 카테고리가 있다면 각자 넣어주세요.
#다른 방식의 라벨링 셋 설정 관련 의견 주셔도 괜찮습니다.

#청결

san_pos_keywords = ['깔끔','깨끗','청결','위생','밝은']
san_neg_keywords = ['더럽','불결','비위생']
san_neu_keywords = ['정리 정돈']

#서비스(고객 만족도) 
#음식이 카테고리에 들어간 건 티나 커피, 토스트 등 
#장기간 머리를 하는 손님에게 서비스 차원에서 음식을 제공하는 미용실(서비스가 좋은 미용실이라 판단)이 있음
cs_pos_keywords = ['웰컴티','배려','토스트','커피','녹차','감동','보조 배터리','마사지','루이보스','홍차'
                  ,'신속','차','스낵','챙겨','쾌적','편안','친절','라떼','아이스티','헤어마사지'
                  ,'아메리카노','쥬스','주스','과자','무료주차권','잡지','무료케어','무료마사지','공짜','무료 주차']
cs_neg_keywords = ['불친절','서비스 엉망','신경질','무관심','건방','안절부절','건성','짜증']
cs_neu_keywords = ['서비스 보통','서비스 그냥']

#매장 분위기/인테리어
mood_pos_keywords=['분위기 좋','쾌적','화사','넓직','넓은','모던','고급진','고급스'
                  ,'자연광','통유리','자연채광','아늑','집같은','가족같은','따스한','아기자기']
mood_neg_keywords=['우울','우중충','어둡','협소','허름','넓 않','너저분','불편']
mood_neu_keywords=[]

#가격
price_pos_keywords=['싼','싼 곳','싼곳','혜자스러운','혜자','착한 가격','파격적','가성비 좋은','값 싼','저렴','개이득','핵이득','합리적','훨 저렴','가격 좋',]
price_neg_keywords=['부담','비싼','비쌈','가격이 있는','비싸','부담 스러운','왕부담','핵부담','창 렬', '가격 쎄','비 쌈',]
price_neu_keywords=['평준화','평균적','무난한 가격']

# 위치/교통
loc_pos_keywords = ['가까운','가까운 곳','가까이','위치 좋','접근성 좋','역 근처','걸어서','가깝','가까워']
loc_neg_keywords = ['너무 먼','가기 힘']
loc_neu_keywords = []

# 헤어 관련 만족도(디자이너 실력, 시술 후 헤어 상태 등 헤어 관련 키워드)
sat_pos_keywords =['인스타각','그뤠잇','탱글탱글','빛나는','존예','생동감','흡족','대만족','부들부들','자연스러운','자연스런','자연스'
                   ,'보들보들','갸름해','윤기있는','생기있는','찰랑','트렌디','단정','디테일하게','맛집','디플로마','라인'
                   ,'쩐다','쩌는','개쩌는','믿음가는','흡족한','훈녀','훈남','세련미','여신','기적같은','예쁘','예쁜','이쁘','좋'
                   ,'감격','네임벨류','세심','비단','섬세','색다른','꼼꼼','체크','자연스','금손','찰떡같','멋스러'
                   ,'전문적','전문가','프로페셔널','세련','최신 유행','신뢰받는','높은 만족도','흡족']
sat_neg_keywords=['언벨런스','뻣뻣한','상한','푸석','빗자루','해그리드','부시시','부스스','맘에 안'
                  ,'망한','거지','망함','식겁','돈 아깝','버려','버렸','버림','지저분','붕 뜨']
sat_neu_keywords=[]

# 그 외(카테고리 분류하기 애매한 단어들) 
etc_pos_keywords=['신기','최고','대박','굿','진짜 최고','행복','감격'
                   ,'굿굿','굳','즐겁','편안','훌륭','만족'
                   ,'힐링','웃긴','웃겨','시원','조화','설레','해피','멋진','아낌없는','긍정'
                  ,'짱','진짜 좋','진짜 짱','착한']
etc_neg_keywords=['별로','아쉬운','아쉬움','별거 아닌','별거 없','폭망','감흥 없','비추','개 빡'
                 ,'당황','질색','황당','못해','못한','서운','안타까운','안타깝','안타까웠','게으른'
                 ,'귀찮','심술','식겁']
etc_neu_keywords=['나름','괜찮']